<a href="https://colab.research.google.com/github/blublunalnal/CS230/blob/main/CS230_training_logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/blublunalnal/CS230.git

Cloning into 'CS230'...
remote: Enumerating objects: 446, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 446 (delta 79), reused 122 (delta 54), pack-reused 288 (from 1)
Receiving objects: 100% (446/446), 179.54 MiB | 14.01 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [ ]:
%cd CS230/code
!pip install -r requirements.txt

/content/CS230/code


In [ ]:
%cd new_code

/content/CS230/code/new_code


### Copy data to drive/content/

reading image files from drive folder is very costly; so move to collab's content folder first

In [ ]:
import os
import shutil

# Check available space
disk = shutil.disk_usage("/content")
print(f"Free space: {disk.free / (1024**3):.1f} GB")

# Copy data collab/content folder for faster IO
!cp -r /content/drive/MyDrive/cs230output/images/ /content/patches


Free space: 196.2 GB


### Experiment 1
- note: loss diverages
- both training and validation loss are high

In [ ]:
# !python train_multitask.py \
#     --data_dir_path "/content/patches/" \
#     --backbone "vgg16_bn" \
#     --optimizer "Adam" \
#     --lr 0.001 \
#     --momentum 0.3 \
#     --weight_decay 0.01 \
#     --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
#     --log_name "model_vgg16bn"

2025-11-09 16:19:23.725810: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-09 16:19:23.742959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762705163.764863  147113 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762705163.771409  147113 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762705163.788232  147113 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Experiment 2
- note: severe overfitting
- by Epoch 70;
- training: metasasis loss: 0.049; metasasis accuracy: 0.99
- validation: metasasis loss: 2.3; metasasis accuracy: 0.6
- current setup: classifier dropout: 0.2

In [ ]:
# !python train_multitask.py \
#     --data_dir_path "/content/patches/" \
#     --backbone "vgg16_bn" \
#     --optimizer "Adam" \
#     --lr 0.0001 \
#     --weight_decay 0.001 \
#     --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
#     --log_name "model_vgg16bn_small_learning_rate"

2025-11-09 23:39:39.773140: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-09 23:39:39.790847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762731579.812559  270270 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762731579.819229  270270 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762731579.835964  270270 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Experiment 3
- try increasing L2 strength from 0.001 to 0.01
- current setup: classifier dropout 0.2
- by epoch 42:
- training: Metasasis Loss: 0.2754 | Metasasis AUC: 0.9488
- validation: Metasasis Loss: 0.5990 | Metasasis AUC: 0.8062
- note: still overfitting

In [ ]:
!python train_multitask.py \
    --data_dir_path "/content/patches/" \
    --backbone "vgg16_bn" \
    --optimizer "Adam" \
    --lr 0.0001 \
    --weight_decay 0.01 \
    --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
    --log_name "model_vgg16bn_exp3"

2025-11-10 02:29:53.853801: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-10 02:29:53.871880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762741793.894389  318937 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762741793.901336  318937 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762741793.918260  318937 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Experiment 4
- change backbone to unih-2 freeze
- start with a low lr and weight_decay
- at epoch 3,
- training: the metasasis loss is 0.16 and AUC = 0.98
- validation loss 0.37 and AUC 0.91
- from then on, validation loss starts increasing while training loss continues to increase
- takes 1.5 hr to run 7 epochs

In [ ]:
# !python train_multitask.py \
#     --epoch 50\
#     --data_dir_path "/content/patches/" \
#     --backbone "uni2-h-freeze" \
#     --optimizer "Adam" \
#     --lr 0.0001 \
#     --weight_decay 0.001 \
#     --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
#     --log_name "model_UNI_exp4"

2025-11-10 04:07:44.625671: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-10 04:07:44.643933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762747664.665825  346654 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762747664.672510  346654 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762747664.689363  346654 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Experiment 5
- backbone: unih-2 freeze
- use the same lr as exp #4 but a much higher weight-decay

In [ ]:
!python train_multitask.py \
    --epoch 50\
    --data_dir_path "/content/patches/" \
    --backbone "uni2-h-freeze" \
    --optimizer "Adam" \
    --lr 0.0001 \
    --weight_decay 0.015 \
    --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
    --log_name "model_UNI_exp5"

2025-11-10 06:23:49.753875: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-10 06:23:49.771871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762755829.794346  381328 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762755829.801201  381328 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762755829.818190  381328 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Experiment 6
Same setup as exp #5 but twice the dropout rate and weight decay

In [ ]:
!python train_multitask.py \
    --epoch 20\
    --data_dir_path "/content/patches/" \
    --backbone "uni2-h-freeze" \
    --optimizer "Adam" \
    --lr 0.0001 \
    --dropout 0.5 \
    --weight_decay 0.03 \
    --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
    --log_name "model_UNI_exp6"

2025-11-12 18:43:39.376951: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 18:43:39.394525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762973019.416636  189962 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762973019.423321  189962 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762973019.440309  189962 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Experiment 7
- use image data only
- same dropout probability and weight decay as e

In [ ]:
!python train_multitask.py \
    --epoch 15\
    --data_dir_path "/content/patches/" \
    --backbone "uni2-h-freeze" \
    --optimizer "Adam" \
    --lr 0.0001 \
    --image_only \
    --weight_decay 0.015 \
    --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
    --log_name "model_UNI_exp7"

2025-11-12 23:37:34.656955: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 23:37:34.675365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762990654.697832  265910 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762990654.704681  265910 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762990654.721807  265910 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### Experiment 8
- Exp #5 shows quick convergence, want to know if it's the effect of the feature extractor or the double-branch multi-task
- same setup as exp #5 but only have 1 branch - only predicting metasasis

In [ ]:
!python train_singletask.py \
    --epoch 20\
    --data_dir_path "/content/patches/" \
    --backbone "uni2-h-freeze" \
    --optimizer "Adam" \
    --lr 0.0001 \
    --weight_decay 0.015 \
    --log_dir_path "/content/drive/MyDrive/cs230output/training_output/" \
    --log_name "model_UNI_exp8"

2025-11-13 05:55:32.709697: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 05:55:32.728617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763013332.751270  359330 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763013332.758011  359330 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763013332.775498  359330 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 